In [7]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

### Import & Filter Data

In [8]:
# Define Local Authority
LocalAuthority = '"Stirling"'
# Load Family Groups data file and extract the two relevant Family Groups for the selected Local Authority
FamilyGroups = pd.read_csv("Data Files//Family Groups.csv")
FamilyGroup_EnvEco = FamilyGroups.query(f'Local_Authority == {LocalAuthority} & Type == "Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators"').iloc[0].Family_Group
FamilyGroup_ChildHouse = FamilyGroups.query(f'Local_Authority == {LocalAuthority} & Type == "Children, Social Work and Housing indicators"').iloc[0].Family_Group

# Load Family Group Averages data file, load and merge this with the indicator data and then use the family group types above to filter the averages for the appropriate Code and Family Group combinations
FamilyGroupData = pd.read_csv("Data Files//Family Averages.csv")
IndicatorInfo = pd.read_csv("Data Files//Indicator Information.csv")
AlternateLines = pd.read_csv("Data Files//Alternative Lines.csv")
AlternateLines = AlternateLines[['Key_CodePeriod', 'LineType']]
FamilyGroupData = FamilyGroupData.merge(IndicatorInfo[['Code_Sortable', 'FamilyGrouping']], how='left', left_on=['Code'], right_on=['Code_Sortable'], suffixes=('_FGAverages', '_Info'))
FamilyGroupData_SelectedLA = FamilyGroupData.query(f'(FamilyGrouping == "Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators" & Family_Group == "{FamilyGroup_EnvEco}" | FamilyGrouping == "Children, Social Work and Housing indicators" & Family_Group == "{FamilyGroup_ChildHouse}") & `Data Type` == "Real_Annual"')
FamilyGroupData_SelectedLA = FamilyGroupData_SelectedLA[['Code', 'Period', 'Value_FG_Mean', 'Value_FG_Median']]
FamilyGroupData_SelectedLA['Key_CodePeriod'] = FamilyGroupData_SelectedLA['Code'] + FamilyGroupData_SelectedLA['Period']
FamilyGroupData_SelectedLA = FamilyGroupData_SelectedLA.merge(AlternateLines, how='left', on='Key_CodePeriod', suffixes=('_Family', '_Alternate'))
FamilyGroupData_SelectedLA = FamilyGroupData_SelectedLA[['Code', 'Period', 'Value_FG_Mean', 'Value_FG_Median', 'LineType']]

# Load Indicator Data and filter it for the selected Local Authority
IndicatorData = pd.read_csv("Data Files//Indicator Data.csv")
IndicatorData_SelectedLA = IndicatorData.query(f'LocalAuthority == {LocalAuthority} & `Data Type` == "Real_Annual"')[['Code', 'Period', 'Value', 'FamilyRank']]
IndicatorData_SelectedLA['Key_CodePeriod'] = IndicatorData_SelectedLA['Code'] + IndicatorData_SelectedLA['Period']
IndicatorData_SelectedLA = IndicatorData_SelectedLA.merge(AlternateLines, how='left', on='Key_CodePeriod', suffixes=('_Indicator', '_Alternate'))
IndicatorData_SelectedLA = IndicatorData_SelectedLA[['Code', 'Period', 'Value', 'FamilyRank', 'LineType']]
IndicatorData_SelectedLA

# Print column names as reference
print("IndicatorData_SelectedLA")
print(list(IndicatorData_SelectedLA.columns.values))
print("FamilyGroupData_SelectedLA")
print(list(FamilyGroupData_SelectedLA.columns.values))
print("IndicatorInfo")
print(list(IndicatorInfo.columns.values))

IndicatorData_SelectedLA
['Code', 'Period', 'Value', 'FamilyRank', 'LineType']
FamilyGroupData_SelectedLA
['Code', 'Period', 'Value_FG_Mean', 'Value_FG_Median', 'LineType']
IndicatorInfo
['Title', 'Code', 'Code_Sortable', 'ReportingPeriod', 'MeasureType', 'NumberFormat', 'YMin', 'YMax', 'ISCategory', 'Committee', 'FamilyGrouping', 'StirlingService', 'Ranking_Type', 'NumberFormat_NoText', 'Source', 'Numerator_Correct', 'Denominator_Correct', 'Numerator_Match', 'Denominator_Match', 'Numerator_Multipier', 'Denominator_Multiplier', 'Ranking_GoldilocksMidpoint', 'NumberFormat_Axis', 'Format_Python', 'FormatAxis_Python', 'AdditionalAxisDenominator_Python', 'FormatAxis_Plotly_Prefix', 'FormatAxis_Plotly_Suffix', 'ImgPxlWidth_Plotly', 'ImgPxlHeight_Plotly', 'SubGroup_PythonReport', 'YMin_Plotly', 'YMax_Plotly', 'OData__ColorTag']


### Fix Data Series Gaps by Copying First row of each alternate LineType

In [9]:
def addGaps(df) :
    df.sort_values(by=['Code','Period'], inplace=True)
    df.reset_index(drop=True)
    df['LineType'] = df['LineType'].fillna('')
    previousrow = {}
    additions = []
    for k, row in df.iterrows() :
        if len(previousrow) == 0 :
            previousrow = row
        elif previousrow.LineType != row.LineType and previousrow.Code == row.Code :
            addition = previousrow
            addition.LineType = row.LineType
            additions.append(addition)
            previousrow = row
        else :
            previousrow = row
    # additions = pd.DataFrame(additions).reset_index(drop=True)
    df = df.append(additions, ignore_index=True)
    df.sort_values(by=['Code', 'Period'], inplace=True)
    df.reset_index(drop=True)
    return df
FamilyGroupData_SelectedLA = addGaps(FamilyGroupData_SelectedLA)
IndicatorData_SelectedLA = addGaps(IndicatorData_SelectedLA)

### Define Graph creation functions

In [10]:


# Define function to extract line series information filtered for the identified indicator code from the dataframes and combines into a single dataframe for the purpose of graphing.
def extractLineSeries (Code, LAData = IndicatorData_SelectedLA,FGData = FamilyGroupData_SelectedLA) :
    IndiData = LAData.query(f'Code == "{Code}"')[['Period','Value','FamilyRank','LineType']]
    IndiData['FormattedFamilyRank'] = "#" + IndiData['FamilyRank'].map(int).map(str)
    FGData = FGData.query(f'Code == "{Code}"')[['Period', 'Value_FG_Median']]
    Data = FGData.merge(IndiData,how = 'left', on = ['Period'], suffixes = ('_FG','_Indi'))
    return Data


def addSpecificTrace(Color, DataSlice_Y, DataSlice_X, Name, LineType,Fill,ShowLegend, Chart):
    Chart.add_trace(
        go.Scatter(
            x=DataSlice_X,
            y=DataSlice_Y,
            name=Name,
            mode='lines',
            fill = Fill,
            showlegend = ShowLegend,
            line=dict(
                color=Color,
                width=4,
                shape='spline',
                smoothing=0.7,
                dash=LineType
            )
        )
    )
    return Chart

# Define function to create a standard styled line chart for a given set of data.
def createLineChart(Data,Format,Prefix,Suffix,Ymax,Ymin) :
    Chart = go.Figure()
    # Add local authority values as area line.
    addSpecificTrace('#5d2381', Data.query(f'LineType == ""')['Value'], Data['Period'], LocalAuthority, None,'tozeroy',True, Chart)
    addSpecificTrace('#5d2381', Data.query(f'LineType == "Dashed"')['Value'], Data.query(f'LineType == "Dashed"')['Period'], "DashedLineLA", "dash", 'tozeroy',False, Chart)
    addSpecificTrace('#23815d', Data.query(f'LineType == ""')['Value_FG_Median'], Data['Period'], "Family Group Median", None, None, True, Chart)
    addSpecificTrace('#23815d', Data.query(f'LineType == "Dashed"')['Value_FG_Median'], Data.query(f'LineType == "Dashed"')['Period'], "DashedLineFG", "dash", None, False, Chart)
    # Format chart
    Chart.update_layout(
        xaxis_type ='category',
        plot_bgcolor =  'rgba(0, 0, 0, 0)',
        width = 700,
        height = 400,
        font = 
            dict(
                    family = "Segoe UI Semibold",
                    color = "black"
                ),
        margin = 
            dict(
                    b = 10,
                    l = 10,
                    r = 10,
                    t = 10,
                    pad = 10
                ),
        legend =
            dict(
                orientation = 'h',
                xanchor = 'left',
                x = 0,
                yanchor = 'top',
                y = 1.1,
                bgcolor = 'rgba(0, 0, 0, 0)',
                itemsizing = 'constant',
                font =
                    dict(
                        size = 15,
                    )
            ),
        yaxis = 
            dict(
                gridcolor = '#ededed',
                tickformat = Format,
                tickprefix = Prefix,
                ticksuffix = Suffix,
                tickfont = dict(size = 15),
                range = [Ymin,Ymax]
            ),
        xaxis = dict(
            tickfont = dict(size = 15)
        )
    )
    return Chart




## Create Graphs using Functions for each Indicator Code

In [11]:
LocalAuthority = 'Stirling'
for row in IndicatorInfo.itertuples() :
    # Filter main dataframes for particular indicator data and separate into arrays
    Code = row.Code_Sortable
    Format = row.FormatAxis_Python
    Ymax = row.YMax_Plotly
    Ymin = row.YMin_Plotly
    if pd.isnull(row.FormatAxis_Plotly_Prefix) :
        Prefix = ""
    else :
        Prefix = row.FormatAxis_Plotly_Prefix
    if pd.isnull(row.FormatAxis_Plotly_Suffix) :
        Suffix = ""
    else :
        Suffix = row.FormatAxis_Plotly_Suffix
    #Format = row.Format_Python
    Data = extractLineSeries(Code)
    # Create line charts and export these to png
    LineChart = createLineChart(Data,Format, Prefix, Suffix, Ymax, Ymin)
    LineChart.write_image(f'Graphs//{Code}_Line.svg', width = f'{row.ImgPxlWidth_Plotly}', height = f'{row.ImgPxlHeight_Plotly}')

#### Show Graph Examples

In [12]:
LineChart.show()